# MATH50003 Numerical Analysis: Problem 3

This problem sheet explores implementation of triangular solves,
supporting a matrix with two super-diagonals, as well as
permutation and Householder reflections that can be applied to a vector in
$O(n)$ complexity.

In [1]:
using LinearAlgebra, Test

# We will override these functions below
import Base: getindex, setindex!, size, *, \

## 1. Dense Matrices

**Problem 1.1** Show that `A*x` is not
implemented as `mul(A, x)` from the lecture notes
by finding a `Float64` example  where the bits do not match.




## 2. Triangular Matrices

**Problem 2.1** Complete the following functions for solving linear systems with
triangular systems by implementing back and forward-substitution:

In [2]:
function ldiv(U::UpperTriangular, b)
    n = size(U,1)
    
    if length(b) != n
        error("The system is not compatible")
    end
        
    x = zeros(n)  # the solution vector
    ## TODO: populate x using back-substitution

    
    for k = n:-1:1  # start with k=n, then k=n-1, ...
        r = b[k]  # dummy variable
        for j = k+1:n
            r -= U[k,j]*x[j] # equivalent to r = r-U[k,j]*x[j]
        end
        x[k] = r/U[k,k]
    end
    x
end

function ldiv(U::LowerTriangular, b)
    n = size(U,1)
    
    if length(b) != n
        error("The system is not compatible")
    end
        
    x = zeros(n)  # the solution vector
    ## TODO: populate x using forward-substitution
end

ldiv (generic function with 2 methods)

**Problem 2.2⋆** Given $𝐱 ∈ ℝ^n$, find a lower triangular matrix of the form
$$
L = I - 2 𝐯 𝐞_1^⊤
$$
such that:
$$
L 𝐱 = x_1 𝐞_1.
$$
What does $L𝐲$ equal if $𝐲  ∈ ℝ^n$ satisfies $y_1 = 𝐞_1^⊤ 𝐲 = 0$?

## 3. Banded matrices

**Problem 3.1** Complete the implementation of `UpperTridiagonal` which represents a banded matrix with
bandwidths $(l,u) = (0,2)$:

In [3]:
struct UpperTridiagonal{T} <: AbstractMatrix{T}
    d::Vector{T}   # diagonal entries
    du::Vector{T}  # super-diagonal enries
    du2::Vector{T} # second-super-diagonal entries
end

size(U::UpperTridiagonal) = (length(U.d),length(U.d))

function getindex(U::UpperTridiagonal, k::Int, j::Int)
    d,du,du2 = U.d,U.du,U.du2
    # TODO: return U[k,j]
end

function setindex!(U::UpperTridiagonal, v, k::Int, j::Int)
    d,du,du2 = U.d,U.du,U.du2
    if j > k+2
        error("Cannot modify off-band")
    end

    # TODO: modify d,du,du2 so that U[k,j] == v

    U # by convention we return the matrix
end

setindex! (generic function with 90 methods)

**Problem 3.2** Complete the following implementations of `*` and `\` for `UpperTridiagonal` so that
they take only $O(n)$ operations.

In [4]:
function *(U::UpperTridiagonal, x::AbstractVector)
    T = promote_type(eltype(U), eltype(x)) # make a type that contains both the element type of U and x
    b = zeros(T, size(U,1)) # returned vector
    # TODO: populate b so that U*x == b (up to rounding)
end

function \(U::UpperTridiagonal, b::AbstractVector)
    T = promote_type(eltype(U), eltype(b)) # make a type that contains both the element type of U and b
    x = zeros(T, size(U,2)) # returned vector
    # TODO: populate x so that U*x == b (up to rounding)
end

\ (generic function with 136 methods)

## 4. Permutations

**Problem 4.1⋆** What are the permutation matrices corresponding to the following permutations?
$$
\begin{pmatrix}
1 & 2 & 3 \\
3 & 2 & 1
\end{pmatrix}, \begin{pmatrix}
1 & 2 & 3 & 4 & 5 & 6\\
2 & 1 & 4 & 3 & 6 & 5
\end{pmatrix}.
$$

**SOLUTION**

Let $\sigma=\begin{pmatrix}1 & 2 & 3 \\ 3 & 2 & 1\end{pmatrix}$ and $\tau=\begin{pmatrix}1 & 2 & 3 & 4 & 5 & 6\\ 2 & 1 & 4 & 3 & 6 & 5\end{pmatrix}$. There are two ways to construct $P_\sigma$ and $P_\tau$.

- Column by column:
  $$P_\sigma=
  \left(\mathbf{e}_{\sigma_1^{-1}}\middle|\mathbf{e}_{\sigma_2^{-1}}\middle|\mathbf{e}_{\sigma_3^{-1}}\right)=
  \left(\mathbf{e}_3\middle|\mathbf{e}_2\middle|\mathbf{e}_1\right)$$
  $$P_\tau=
  \left(\mathbf{e}_{\tau_1^{-1}}\middle|\mathbf{e}_{\tau_2^{-1}}\middle|\mathbf{e}_{\tau_3^{-1}}\middle|\mathbf{e}_{\tau_4^{-1}}\middle|\mathbf{e}_{\tau_5^{-1}}\middle|\mathbf{e}_{\tau_6^{-1}}\right)=
  \left(\mathbf{e}_2\middle|\mathbf{e}_1\middle|\mathbf{e}_4\middle|\mathbf{e}_3\middle|\mathbf{e}_6\middle|\mathbf{e}_5\right)$$
- Row by row:
  $$P_\sigma=I[\mathbf{\sigma},:]=
  \begin{pmatrix}
  I[\sigma_1,:]\\
  I[\sigma_2,:]\\
  I[\sigma_3,:]
  \end{pmatrix}=
  \begin{pmatrix}
  I[3,:]\\
  I[2,:]\\
  I[1,:]
  \end{pmatrix}$$
  $$P_\tau=I[\mathbf{\tau},:]=
  \begin{pmatrix}
  I[\tau_1,:]\\
  I[\tau_2,:]\\
  I[\tau_3,:]\\
  I[\tau_4,:]\\
  I[\tau_5,:]\\
  I[\tau_6,:]\\
  \end{pmatrix}=
  \begin{pmatrix}
  I[2,:]\\
  I[1,:]\\
  I[4,:]\\
  I[3,:]\\
  I[6,:]\\
  I[5,:]\\
  \end{pmatrix}$$

Either way, we have
$$P_\sigma=
\begin{pmatrix}
0 & 0 & 1\\ 
0 & 1 & 0\\ 
1 & 0 & 0
\end{pmatrix}
\qquad\text{and}\qquad
P_\tau=
\begin{pmatrix}
0 & 1 & 0 & 0 & 0 & 0 \\
1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 \\
0 & 0 & 0 & 0 & 1 & 0
\end{pmatrix}$$


**Problem 4.2** Complete the implementation of a type representing
permutation matrices that supports `P[k,j]` and such that `*` takes $O(n)$ operations.

In [7]:
using Test, LinearAlgebra

struct PermutationMatrix <: AbstractMatrix{Int}
    p::Vector{Int} # represents the permutation whose action is v[p]
    function PermutationMatrix(p::Vector)
        sort(p) == 1:length(p) || error("input is not a valid permutation")
        new(p)
    end
end

Base.size(P::PermutationMatrix) = (length(P.p),length(P.p))

function Base.getindex(P::PermutationMatrix, k::Int, j::Int)
    # TODO: Return P[k,j]
    P.p[k] == j ? 1 : 0 
end

function Base.:*(P::PermutationMatrix, x::AbstractVector)
    # TODO: permute the entries of x
    x[P.p]
end

# If your code is correct, this "unit test" will succeed
p = [1, 4, 2, 5, 3]
P = PermutationMatrix(p)
@test P == I(5)[p,:]

Test Passed
  Expression: P == (I(5))[p, :]
   Evaluated: [1 0 … 0 0; 0 0 … 1 0; … ; 0 0 … 0 1; 0 0 … 0 0] == sparse([1, 3, 5, 2, 4], [1, 2, 3, 4, 5], Bool[1, 1, 1, 1, 1], 5, 5)

The following codes show that `*` takes $O(n)$ of both time and space.

In [6]:
using BenchmarkTools, Random
x=0 # for some reason, an error "UndefVarError: x not defined" will occur without this line. p and P are previously defined so they work fine.
for n = 10 .^ (1:7)
    print("n = ", n)
    p = randperm(n)
    P = PermutationMatrix(p)
    x = randn(n)
    @btime P*x
end

n = 10  63.673 ns (1 allocation: 144 bytes)
n = 100  147.771 ns (1 allocation: 896 bytes)
n = 1000  1.030 μs (1 allocation: 7.94 KiB)
n = 10000  10.300 μs (2 allocations: 78.17 KiB)
n = 100000  139.600 μs (2 allocations: 781.30 KiB)
n = 1000000  10.386 ms (2 allocations: 7.63 MiB)
n = 10000000  126.302 ms (2 allocations: 76.29 MiB)


## 5. Orthogonal matrices

**Problem 5.1⋆** Show that orthogonal matrices preserve the 2-norm of vectors:
$$
\|Q 𝐯\| = \|𝐯\|.
$$

**SOLUTION**
$$
\|Q 𝐯\|^2 = (Q 𝐯)^⊤ Q 𝐯 = 𝐯^⊤ Q^⊤ Q 𝐯 = 𝐯^⊤  𝐯 = \|𝐯\|^2
$$
∎

**Problem 5.2⋆** Show that the eigenvalues $λ$ of an orthogonal matrix $Q$ are
on the unit circle: $|λ| = 1$.

**SOLUTION**

Let $𝐯$ be a unit eigenvector corresponding to $λ$: $Q 𝐯 = λ 𝐯$ with $\|𝐯\| = 1$. Then
$$
1 = \| 𝐯 \| = \|Q 𝐯 \| =  \| λ 𝐯 \| = |λ|.
$$
∎

**Problem 5.3⋆** Explain why an orthogonal matrix $Q$ must be equal to $I$ if all its eigenvalues are 1.

**SOLUTION**

Let $Q$'s linearly independent unit eigenvectors be $\mathbf{v}_1,\dots,\mathbf{v}_n$, then $Q\mathbf{v}_k=\mathbf{v}_k$ for all $k$ since all $Q$'s eigenvalues are $1$.

Now if we let $V=\left(\mathbf{v}_1\middle|\dots\middle|\mathbf{v}_n\right)$, then $QV=V$. Recalling that vectors in $V$ are linearly independent, which means that $V$ is invertible, we can multiply both sides by $V^{-1}$ to get $Q=I$.


**Problem 5.4** Complete the implementation of a type representing
reflections that supports `Q[k,j]` and such that `*` takes $O(n)$ operations.

In [10]:
# Represents I - 2v*v'
struct Reflection{T} <: AbstractMatrix{T}
    v::Vector{T}
end

Reflection(x::Vector{T}) where T = Reflection{T}(x/norm(x))

Base.size(Q::Reflection) = (length(Q.v),length(Q.v))

function Base.getindex(Q::Reflection, k::Int, j::Int)
    # TODO: Return Q[k,j]
    (k == j) - 2Q.v[k]*Q.v[j]
end

function Base.:*(Q::Reflection, x::AbstractVector)
    # TODO: permute the entries of x
    x - 2*Q.v * dot(Q.v,x)
end

# If your code is correct, these "unit tests" will succeed
x = randn(5)
Q = Reflection(x)
v = x/norm(x)
@test Q == I-2v*v'
@test Q*v ≈ -v
@test Q'Q ≈ I

Test Passed
  Expression: Q' * Q ≈ I
   Evaluated: [1.0 -1.734723475976807e-18 … -2.7755575615628914e-17 -2.7755575615628914e-17; -1.734723475976807e-18 0.9999999999999998 … -3.469446951953614e-18 1.734723475976807e-18; … ; -2.7755575615628914e-17 -3.469446951953614e-18 … 0.9999999999999998 -5.551115123125783e-17; -2.7755575615628914e-17 1.734723475976807e-18 … -5.551115123125783e-17 0.9999999999999998] ≈ UniformScaling{Bool}(true)

The following codes show that `*` takes $O(n)$ of both time and space.

In [14]:
using BenchmarkTools, Random
for n = 10 .^ (1:7)
    print("n = ", n)
    x = randn(n)
    Q = Reflection(x)
    v = randn(n)
    @btime Q*v
end

n = 10  150.841 ns (3 allocations: 432 bytes)
n = 100  263.036 ns (3 allocations: 2.62 KiB)
n = 1000  2.156 μs (3 allocations: 23.81 KiB)
n = 10000  20.500 μs (6 allocations: 234.52 KiB)
n = 100000  298.900 μs (6 allocations: 2.29 MiB)
n = 1000000  6.157 ms (6 allocations: 22.89 MiB)
n = 10000000  64.061 ms (6 allocations: 228.88 MiB)


**Problem 5.5** Complete the following implementation of a Housholder reflection  so that the
unit tests pass.

In [31]:
householderreflection(s::Bool, x::AbstractVector) = Reflection(x-ifelse(s,one(eltype(x)),-one(eltype(x)))*[norm(x);zeros(length(x)-1)])

x = randn(5)
Q = householderreflection(true, x)
@test Q isa Reflection
@test Q*x ≈ norm(x)*[1; zeros(length(x)-1)]

Q̃ = householderreflection(false, x)
@test Q̃ isa Reflection
@test Q̃*x ≈ -norm(x)*[1; zeros(length(x)-1)]

Test Passed
  Expression: Q̃ * x ≈ -(norm(x)) * [1; zeros(length(x) - 1)]
   Evaluated: [-2.7674271803090424, 0.0, 0.0, 0.0, 0.0] ≈ [-2.7674271803090424, -0.0, -0.0, -0.0, -0.0]